### This script is taking all the acknowledgement text, processing it, and then calling paraphrasing model to perform text based Augmentation. Then save the result as a image(.png) format. Finally, it goes through all the images and perform image transformation and save the result as .png format.

In [1]:
## importing the modules from ETDAugmentation script
from ipynb.fs.full.ETDaugmentation import *
from imgaug import augmenters as iaa
import imgaug as ia
import pandas as pd
import numpy as np
import cv2
import glob
import re
import time
from tqdm import tqdm
import tensorflow as tf

2023-07-13 14:29:06.686121: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 14:29:08.845745: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("GPU device not found")
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


2023-07-13 15:07:32.536655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-13 15:07:36.300437: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /device:GPU:0 with 11523 MB memory:  -> device: 0, name: Tesla V100-SXM2-16GB, pci bus id: 0000:1a:00.0, compute capability: 7.0


In [13]:
class ACK_TEXT_IMG_AUG:
    
    def list_of_phrases(self, phrases_ack):
        phrases_list = []
        self.phrases_ack = phrases_ack
        for text in tqdm(self.phrases_ack, desc = 'Processing list of phrases'):
            ack = parser.preprocess_ack(text)
            ack_paraphrase = augmentation.paraphrased_text(ack)
            ack_list = [ack_paraphrase]
            phrases_list.append(ack_list)
        
        return phrases_list

    ## This function is taking all the augmented text and wrapping it so that it can fit into the page
    ## it also adds heading to the top of the page

    def list_of_ack(self, ack_text):
        list_ack = []
        self.ack_text = ack_text
        for text in tqdm(self.ack_text ,desc = 'Processing list of ack'):
            for row in text:
                ack_text_wrap = augmentation.wrap_text(row)
                ack_title = "\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\tACKNOWLEDGEMENTS\n" + ack_text_wrap
                list_ack.append(ack_title)
        return list_ack
    
    def text_on_img(self, fileName, text, size):
        self.fileName = fileName
        self.text = text
        self.size = size
        self.W = 2360
        self.H = 3200
        self.w = 2300
        self.h = 3140
        font = ImageFont.truetype('NimbusMonoPS-Bold.otf', size)
        image = Image.new(mode = "RGB", size = (self.W, self.H), color = "white")
        draw = ImageDraw.Draw(image)
        w, h = draw.textsize(text)
        ## Adjust the texual position in a page and draw the text on a image
        '''
        dedication -- 14 width and height 8; 
        ack -- 15 width and 6 height; 
        gabs -- 13 width and height 16
        abstract -- 16 width and height 12
        chapter abstract -- 18 width and height 6
        '''
        draw.text(((self.W-w)/15,(self.H-h)/6), text, font=font, fill=(0,0,0), spacing=60)
        image.save(fileName)
        
        return image
    
    ## This function is calling text_on_image function and save the image to the directory
    def ack_aug(self, ack_text_title):
        self.ack_text_title = ack_text_title
        for n, row in tqdm(enumerate(self.ack_text_title),desc = 'Processing augmented images'):
            fileName = ("/home/mchou001/Label-Acknowledgement/aug_text{}.png".format(n))
            save_image = self.text_on_img(fileName, row, 40) ## dedication -- 43, ack and abstract - 40, gabs -- 38
        
        return save_image
    
    def img_transform(self, img_files):
        self.img_files = img_files
        
        img_augmentation = iaa.Sequential([
            #iaa.Affine(rotate=(-6, 6)),
            #iaa.AdditiveGaussianNoise(scale=(10, 50)), #used
            #iaa.SaltAndPepper(p=0.01), #used
            #iaa.GaussianBlur(sigma=0.5), #used
            #iaa.LinearContrast(alpha=1),#used
            #iaa.PerspectiveTransform(scale=0.025, keep_size=True)
        ], random_order = True)
        
        
        for n, images in tqdm(enumerate(self.img_files[0:]),desc = 'Processing augmented text images'):
            aug_image = img_augmentation(image=cv2.imread(images))
            cv2.imwrite('./Label-Acknowledgement/515/aug{}.png'.format(n),aug_image)
        
        # lower = 0
        # upper = 1000
        # while lower < upper:
            # for n, images in enumerate(self.img_files[0:]):
            #     aug_image = img_augmentation(image=cv2.imread(images))
            #     cv2.imwrite('./Label-Acknowledgement/501/aug{}.png'.format(lower),aug_image)
            #     lower += 1

In [14]:
if __name__ == "__main__":

    start = time.time()
    with tf.device('/gpu:0'):
        parser = Preprocessor()
        augmentation = Augmentation()

        acknowledgement_text = ACK_TEXT_IMG_AUG()

        etd_data = pd.read_csv('ETD_aug.csv')
        etd_data.set_index("class", inplace = True)

        label_ack = etd_data.loc["Label-Acknowledgement"]
        phrases_ack = label_ack['text']

        ack = acknowledgement_text.list_of_phrases(phrases_ack)


        ack_ = acknowledgement_text.list_of_ack(ack)
        df = pd.DataFrame(ack_, columns = ['text'])
        df.to_csv('./Label-Acknowledgement/acknowledgement-v3.csv', encoding = 'utf-8', index= None)


        ack_text_aug = acknowledgement_text.ack_aug(ack_)

        img_path = sorted(glob.glob('./Label-Acknowledgement/*.png'), key = parser.numericalSort)

        augmented_img = acknowledgement_text.img_transform(img_path)
    
    stop = time.time()
    print(f'Generating took: {(stop-start)/60} minutes')

Processing list of ack: 100%|██████████| 543/543 [00:00<00:00, 3964.96it/s]
Processing augmented images: 543it [04:59,  1.81it/s]
Processing augmented text images: 543it [01:56,  4.65it/s]

Generating took: 37.600044127305345 minutes


In [3]:
import os

# folder path
dir_path = './Label-Acknowledgement/515'
count = 0
# Iterate directory
for path in os.listdir(dir_path):
    # check if current path is a file
    if os.path.isfile(os.path.join(dir_path, path)):
        count += 1
print('File count:', count)

File count: 543


In [4]:
parser = Preprocessor()
image_path = sorted(glob.glob('/home/mchou001/Label-Acknowledgement/515/*.png'), key = parser.numericalSort)
df = pd.DataFrame(image_path, columns = ['img_path'])
df.to_csv('./Label-Acknowledgement/ack_img_aug-v3.csv', encoding = 'utf-8', index= None)

In [5]:
df1 = pd.read_csv('./Label-Acknowledgement/ack_img_aug-v3.csv')
df2 = pd.read_csv('./Label-Acknowledgement/acknowledgement-v3.csv')
df1

,img_path
0,/home/mchou001/Label-Acknowledgement/515/aug0.png
1,/home/mchou001/Label-Acknowledgement/515/aug1.png
2,/home/mchou001/Label-Acknowledgement/515/aug2.png
3,/home/mchou001/Label-Acknowledgement/515/aug3.png
4,/home/mchou001/Label-Acknowledgement/515/aug4.png
...,...
538,/home/mchou001/Label-Acknowledgement/515/aug53...
539,/home/mchou001/Label-Acknowledgement/515/aug53...
540,/home/mchou001/Label-Acknowledgement/515/aug54...
541,/home/mchou001/Label-Acknowledgement/515/aug54...


In [6]:
df3 = [txt.strip('\t') for txt in df2['text']]
df4 = pd.DataFrame(df3, columns = ['text'])
df4['class'] = np.nan
df4 = df4.fillna("Label-Acknowledgement")
df4.to_csv('./Label-Acknowledgement/updated_ack-v3.csv', encoding = 'utf-8', index= None)
df4

,text,class
0,ACKNOWLEDGEMENTS\nDr. C.R. Bell gave me valuab...,Label-Acknowledgement
1,ACKNOWLEDGEMENTS\nThe author would like to exp...,Label-Acknowledgement
2,ACKNOWLEDGEMENTS\nThe author thanks Professor ...,Label-Acknowledgement
3,ACKNOWLEDGEMENTS\n3. The research was sponsore...,Label-Acknowledgement
4,ACKNOWLEDGEMENTS\nI am indebted to Professor H...,Label-Acknowledgement
...,...,...
538,ACKNOWLEDGEMENTS\nThe author would like to tha...,Label-Acknowledgement
539,ACKNOWLEDGEMENTS\nThe author would like to tha...,Label-Acknowledgement
540,ACKNOWLEDGEMENTS\nACKNOWLEDGEMENTS The author ...,Label-Acknowledgement
541,ACKNOWLEDGEMENTS\nThe author wishes to thank h...,Label-Acknowledgement


In [7]:
df5 = pd.read_csv('./Label-Acknowledgement/updated_ack-v3.csv')
df6 = pd.concat([df1, df4], axis = 1)
df6.to_csv('./Label-Acknowledgement/ack-v3.csv', encoding = 'utf-8', index= None)

In [8]:
df6

,img_path,text,class
0,/home/mchou001/Label-Acknowledgement/515/aug0.png,ACKNOWLEDGEMENTS\nDr. C.R. Bell gave me valuab...,Label-Acknowledgement
1,/home/mchou001/Label-Acknowledgement/515/aug1.png,ACKNOWLEDGEMENTS\nThe author would like to exp...,Label-Acknowledgement
2,/home/mchou001/Label-Acknowledgement/515/aug2.png,ACKNOWLEDGEMENTS\nThe author thanks Professor ...,Label-Acknowledgement
3,/home/mchou001/Label-Acknowledgement/515/aug3.png,ACKNOWLEDGEMENTS\n3. The research was sponsore...,Label-Acknowledgement
4,/home/mchou001/Label-Acknowledgement/515/aug4.png,ACKNOWLEDGEMENTS\nI am indebted to Professor H...,Label-Acknowledgement
...,...,...,...
538,/home/mchou001/Label-Acknowledgement/515/aug53...,ACKNOWLEDGEMENTS\nThe author would like to tha...,Label-Acknowledgement
539,/home/mchou001/Label-Acknowledgement/515/aug53...,ACKNOWLEDGEMENTS\nThe author would like to tha...,Label-Acknowledgement
540,/home/mchou001/Label-Acknowledgement/515/aug54...,ACKNOWLEDGEMENTS\nACKNOWLEDGEMENTS The author ...,Label-Acknowledgement
541,/home/mchou001/Label-Acknowledgement/515/aug54...,ACKNOWLEDGEMENTS\nThe author wishes to thank h...,Label-Acknowledgement


In [6]:
#!zip -r ack_txt_img.zip ./Label-Acknowledgement/501/

  adding: Label-Acknowledgement/501/ (stored 0%)
  adding: Label-Acknowledgement/501/aug987.png (deflated 15%)
  adding: Label-Acknowledgement/501/aug174.png (deflated 17%)
  adding: Label-Acknowledgement/501/aug884.png (deflated 15%)
  adding: Label-Acknowledgement/501/aug606.png (deflated 15%)
  adding: Label-Acknowledgement/501/aug87.png (deflated 17%)
  adding: Label-Acknowledgement/501/aug809.png (deflated 16%)
  adding: Label-Acknowledgement/501/aug729.png (deflated 15%)
  adding: Label-Acknowledgement/501/aug883.png (deflated 16%)
  adding: Label-Acknowledgement/501/aug365.png (deflated 18%)
  adding: Label-Acknowledgement/501/aug515.png (deflated 16%)
  adding: Label-Acknowledgement/501/aug750.png (deflated 18%)
  adding: Label-Acknowledgement/501/aug759.png (deflated 15%)
  adding: Label-Acknowledgement/501/aug549.png (deflated 18%)
  adding: Label-Acknowledgement/501/aug659.png (deflated 20%)
  adding: Label-Acknowledgement/501/aug284.png (deflated 15%)
  adding: Label-Acknow